In [2]:
import numpy as np
from scipy.integrate import quad
from scipy.optimize import bisect

In [3]:
# ============================
# Constants
# ============================

kpcincm = 3.0857e21       # cm / kpc
MSuninGeV = 1.115e57      # GeV
rSun = 8.277              # kpc
rhoSun = 0.4              # GeV / cm^3
r200 = 220.2              # kpc

In [4]:
# ============================
# DM profile
# ============================

def rhogNFW(r, rhos, rs, gamma):
    x = r / rs
    return rhos / (x**gamma * (1 + x)**(3 - gamma))

def rhocNFWt(r, rhos, rs, rc, n, delta, rt):
    f = np.tanh(r/rc)
    MNFW = 4*np.pi * rhos * rs**3 * (np.log(1+r/rs) - r/rs/(1+r/rs))
    rhocNFW = f**n * rhogNFW(r, rhos, rs, 1.0) + n*f**(n-1) * (1 - f**2) / (4*np.pi*r**2*rc) * MNFW
    if r <= rt:
        return rhocNFW
    else:
        return rhocNFW * (r/rt)**-delta

In [5]:
# ============================
# Local DM density constraint
# ============================

def rhos(rho_func, rs, *args):
    return rhoSun / rho_func(rSun, 1.0, rs, *args)

# ============================
# Halo mass constraint
# ============================

def DeltaM200(rho_func, rs, *args):
    rhos_val = rhos(rho_func, rs, *args)

    def integrand(r):
        return r**2 * rho_func(r, rhos_val, rs, *args)
    
    M200 = 4 * np.pi * quad(integrand, 0, r200, epsrel=1e-5)[0]
    M200 *= kpcincm**3 / MSuninGeV
    return M200 - 1e12

def rs(rho_func, *args):
    return bisect(lambda rs, *args: DeltaM200(rho_func, rs, *args), 0.1, 50, args=(args), xtol=1e-6)

In [18]:
# ============================
# Line-of-sight integral
# ============================

def J_los(theta, rho_func, rhos, rs, *args, D=rSun):
    costh = np.cos(theta)

    def integrand(s):
        r = np.sqrt(D**2 + s**2 - 2*s*D*costh)
        return rho_func(r, rhos, rs, *args)**2

    return kpcincm * quad(integrand, 0, 200.0, epsrel=1e-4)[0]

# ============================
# J-factor
# ============================

def Jfactor(thetamax, rho_func, rhos, rs, *args, D=rSun):

    Nth  = 200
    ths  = np.linspace(0, thetamax, Nth)
    dth  = (ths[-1]-ths[0]) / Nth

    J = 0.0
    for th in ths:
        J += 2*np.pi * np.sin(th) * J_los(th, rho_func, rhos, rs, *args, D=D) * dth

    return J

# ============================
# Compute J-factor
# ============================

thetamax = np.radians(20) # degrees

gamma = 1.26
rsVal = rs(rhogNFW, gamma)
rhosVal = rhos(rhogNFW, rsVal, gamma)
print("J-factor gNFW, gamma=1.26:", Jfactor(thetamax, rhogNFW, rhosVal, rsVal, gamma), 'GeV2/cm5')

J-factor gNFW, gamma=1.26: 2.124397331336515e+23 GeV2/cm5


/var/folders/m5/ptmv1tqj29v601gqzb56cpx80000gn/T/ipykernel_67157/1074882780.py:12: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  return kpcincm * quad(integrand, 0, 200.0, epsrel=1e-4)[0]
